In [6]:
from config import config
from utils import create_ev_bt_feed
from strategies import BuyAndHold, TrailingStop, FixedStop, DRLStrategy
from drl_agent import DRLAgent
import pandas as pd
import backtrader as bt
from backtrader.sizers import PercentSizer
import backtrader.analyzers as btanalyzers
from tensorflow.keras.utils import model_to_dot
import os

## Create reinforcement learning agent and read learnt weights

In [7]:
agent = DRLAgent(train=False)
agent.online_network.load_weights('results/model.weights.h5')
agent.update_target()
model_to_dot(agent.online_network, show_shapes=True, show_layer_names=True).write('charts/model_plot.dot')

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Dense_0 (Dense)                 │ (None, 256)            │         4,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense_1 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 11)             │         2,827 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 72,715 (284.04 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 72,715 (284.04 KB)

...

 Total params: 72,715 (284.04 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 72,715 (284.04 KB)

/opt/miniconda3/envs/master-thesis/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


True

## Load tickers

In [3]:
tickers = pd.read_csv('data/nasdaq_index_list_filtered.csv')['0'].to_list()

## Define benchmark strategies

In [4]:
strategies = {
    'DRL': [DRLStrategy, {'agent': agent, 'train': False}],
    'B&H': [BuyAndHold, {}]
}

for pct in config['stop_pct']:
    strategies[f'FixedSL_{pct:.2f}'] = [FixedStop, {'stop_pct': pct}]
    strategies[f'TrailingSL_{pct:.2f}'] = [TrailingStop, {'stop_pct': pct}]

## Backtrade to validate the strategy

In [5]:
yf_filename = os.path.abspath('data/ohlcv/{}.csv') # Template for filename for a ticker symbol
results_data = []
for key,ticker in enumerate(tickers):
    for strategy_name, strategy in strategies.items():    
        cerebro = bt.Cerebro()
        cerebro.addstrategy(strategy[0], **strategy[1])
        feed, first_close = create_ev_bt_feed(ticker, filename=yf_filename)
        cerebro.adddata(feed)
        value_start = 10 * first_close
        cerebro.broker.setcash(value_start)
        cerebro.addanalyzer(btanalyzers.TimeReturn, _name='return_str', timeframe=bt.TimeFrame.NoTimeFrame)
        cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharpe', riskfreerate=.0)

        # The buy and hold strategy does not have to account for risks associated with the lack of ability of buying back assets.
        cerebro.addsizer(PercentSizer, percents=100.0 if strategy_name == 'B&H' else 90.0)
        cerebro.broker.setcommission(commission=.0)
        cerebro.broker.set_slippage_perc(config['slippage_pct']/100)
        cerebro.broker.set_shortcash(False)
        thestrat = cerebro.run()[0]
        return_pct = list(thestrat.analyzers.return_str.get_analysis().values())[0]*100
        sharpe = thestrat.analyzers.sharpe.get_analysis()['sharperatio']

        # Calculate stop loss level
        if strategy_name == 'DRL':
            # For the DRL strategy map actions to stop levels and calculate the mean
            stops = pd.Series([(.0 if i == 0 else config['stop_pct'][i-1]) for i in thestrat.actions])
            sl_mean = stops.mean()
        elif strategy_name.startswith('FixedSL') or strategy_name.startswith('TrailingSL'):
            # For trailing stop and fixed stop strategies the stop level is always the same
            sl_mean = strategy[1]['stop_pct']
        else:
            # For the buy and hold strategy the stop level is not applicable
            sl_mean = pd.NA
        
        # Store data for a ticker and for a strategy
        results_data.append((strategy_name, ticker, sharpe, return_pct, 
                             value_start, cerebro.broker.get_value(), sl_mean, thestrat.triggs_count))

    # Track progress of the validation
    print("{:<13} | {:<15}".format(
        f"Ticker: {ticker}",
        f"Progress: {(key+1) / len(tickers) * 100:.2f}%"))

# Create dataframe from the list of data, and save the data into a file.
df_results = pd.DataFrame(results_data, columns=['Strategy', 'Ticker', 'Sharpe', 'Return', 'Value_s', 'Value_e', 'SLVL_mean', 'Stop count'])
df_results.set_index(['Strategy', 'Ticker'], inplace=True)
df_results.to_csv('results/results.csv')

Ticker: CPRT  | Progress: 1.04%
Ticker: COST  | Progress: 2.08%
Ticker: AAPL  | Progress: 3.12%
Ticker: AMGN  | Progress: 4.17%
Ticker: CMCSA | Progress: 5.21%
Ticker: INTC  | Progress: 6.25%
Ticker: KLAC  | Progress: 7.29%
Ticker: PCAR  | Progress: 8.33%
Ticker: CTAS  | Progress: 9.38%
Ticker: PAYX  | Progress: 10.42%
Ticker: LRCX  | Progress: 11.46%
Ticker: ADSK  | Progress: 12.50%
Ticker: ROST  | Progress: 13.54%
Ticker: MNST  | Progress: 14.58%
Ticker: MSFT  | Progress: 15.62%
Ticker: ADBE  | Progress: 16.67%
Ticker: FAST  | Progress: 17.71%
Ticker: EA    | Progress: 18.75%
Ticker: CSCO  | Progress: 19.79%
Ticker: REGN  | Progress: 20.83%
Ticker: IDXX  | Progress: 21.88%
Ticker: VRTX  | Progress: 22.92%
Ticker: BIIB  | Progress: 23.96%
Ticker: ODFL  | Progress: 25.00%
Ticker: QCOM  | Progress: 26.04%
Ticker: GILD  | Progress: 27.08%
Ticker: SNPS  | Progress: 28.12%
Ticker: SBUX  | Progress: 29.17%
Ticker: SIRI  | Progress: 30.21%
Ticker: INTU  | Progress: 31.25%
Ticker: MCHP  | Pro